# AutoGrad Engine(Scalar, Reverse-Mode)

A minimal reverse-mode automatic differentiation engine built from scratch for scalar values.

Core idea:
- Forward pass builds a computatuion DAG dynamically
- Backward pass applies the chain rule in reverse topological order

This is NOT symbolic differentiation.
This is NOT numerical differentiation.
This is graph-based reverse-mode autodiff

## Node initialization

Represents a single value in the computation graph.

Each Node stores:
- Value: forward computed scalar value
- grad: accumulated gradient d(output)/d(value)
- parents: nodes this value depends on (graph edges)
- backward_fn: local backward function (closure)

The computation graph is implicit:
If each node knows its parents, the graph already exists.

In [2]:
# Core Structure
class Node:
    def __init__(self, value, parents=(), backward_fn=None):
        self.value = value
        self.grad = 0
        self.parents = parents
        self.backward_fn = backward_fn

## FORWARD Operations (GRAPH CONSTRUCTION)

Each operation:
- computes a forward value
- creates exactly one new Node
- Records dependencies via parents
- Attaches local backward logic as a closure

Important:
- Forward pass ONLY builds the graph
- No gradients are computed here

### add(a, b)

Forward:
- value = a.value + b.value
- parents = (a,b)

Backward (local):
- d(out)/d(a) = 1
- d(out)/d(b) = 1

Gradient contribution is accumulated into parent nodes.

In [3]:
# Function for addition
def add(a, b):
    out = Node(
        value = a.value + b.value,
        parents = (a,b)
    )
    # Gradient Accumulation
    def backward():
        a.grad += out.grad * 1
        b.grad += out.grad * 1
    out.backward_fn = backward
    return out


### mul(a, b)

Forward:
- value = a.value * b.value
- parents = (a,b)

Backward (local):
- d(out)/d(a) = b.value
- d(out)/d(b) = a.value

Uses closure to capture forward values needed for backward.

In [4]:
# Function for multiplication
def mul(a, b):
    out = Node(
        value = a.value * b.value,
        parents = (a,b)
    )
    # Gradient Accumulation
    def backward():
        a.grad += out.grad * b.value
        b.grad += out.grad * a.value
    out.backward_fn = backward
    return out

### Gradient Accumulation

Gradients are accumulated (+=), Not overwritten.

Calling backward multiple times without resetting gradients will accumulate contributions.

This mirrors real frameworks like PyTorch.
Resetting gradient is the user's responsibility.

## post_dfs(node)

Post-order Depth-First Search over the computation DAG.

Guarantees:
- Each node is visited exactly once
- Parents are processed before the node
- Produces a valid topological ordering

Why post-order:
Backward pass requires all downstream gradients to be ready before propagating gradients upstream.

In [5]:
# Function to find node traversal order using Post-DFS
def post_dfs(node, order, visited):
    if node in visited:
        return
    visited.add(node)
    for parent in node.parents:
        post_dfs(parent, order, visited)
    order.append(node)

## BACKWARD PASS (Core Autograd Engine)

Computes gradients for all nodes contributing to the output.

Steps:
1. Seed output gradient: d(out)/d(out) = 1
2. Discover computation graph via post-order DFS
3. Build topological order (dependencies first)
4. Execute backward functions in reverse order

This separates:
- Scheduling (engine)
- Math (local backward functions)

In [6]:
#Backward Pass
def backward(out_node):
    visited, order = set(), []
    post_dfs(out_node, order, visited)
    out_node.grad = 1
    for node in reversed(order):
        if node.backward_fn:
            node.backward_fn()


### Examples

In [7]:
x = Node(5)
y = Node(4)
z = add(x,x)
backward(z)
print(z.grad, x.grad, y.grad) # Outputs are (1 2 0) because x is being used twice so it takes the blame twice.
z = mul(add(x,y),add(x,y))
backward(z)
print(z.grad, x.grad, y.grad) # Outputs are (1 20 18) the answer should be (1 18 18) but due to the gradient not being reset before calling the x node again it added the previous gradient to the new one.

1 2 0
1 20 18


## Zero Gradient (Gradient Reset)

Because gradients accumulate, they must be explicitly reset when starting a new computation.

This is required for:
- Numerical gradient checking
- Training loops
- Repeated backward passes

In [8]:
# Function to reset the grads of all nodes to 0
def zero_grad(output):
    visited, order = set(), []
    post_dfs(output, order, visited)
    for node in order: # Sets the grad of each node to 0
        node.grad = 0

### Example after the implementation of Zero Grad

In [9]:
x = Node(5)
y = Node(4)
z = add(x,x)
backward(z)
print(z.grad, x.grad, y.grad)
zero_grad(z) # Sets the gradients of all the nodes to 0.
z = mul(add(x,y),add(x,y))
backward(z)
print(z.grad, x.grad, y.grad) # Since there's no previous grad stored in the node so it doesn't get accumulated

1 2 0
1 18 18


## Operator Surface (Usability)

To allow natural Python syntax (x + y, 3 - x, etc.), operators are overloaded.

### For add function

In [10]:
def __add__(self, other):
    return add(self, other)

Node.__add__ = __add__

#### Example

In [11]:
x = Node(5)
y = Node(5)
z = x + y
print(z.value)

10


### For mul function

In [12]:
def __mul__(self, other):
    return mul(self, other)

Node.__mul__ = __mul__

#### Example

In [13]:
x = Node(2)
y = Node(5)
z = x * y
print(z.value)

10


### Adding more operations

#### neg(x) — Unary negation

Forward:
- value = a.value * -1
- parents = (a)

Backward:
- d(out)/d(a) = -1

In [14]:
# Function for negation of a single value
def neg(a):
    out = Node(
        value = a.value * -1,
        parents= (a,)
    )
    # Gradient Accumulation
    def backward(): # dy/dx = -1
        a.grad += out.grad * -1
    out.backward_fn = backward
    return out

In [ ]:
def __neg__(self):
    return neg(self)

Node.__neg__ = __neg__

##### Example

In [16]:
x = Node(4)
y = Node(5)
z = x * -y # Composite of ops (First runs the __neg__(self) then does the __mul__(self, other) function)
print(z.value)
backward(z)
print(z.grad, x.grad, y.grad)

-20
1 -5 -4


#### sub(self, other) — Composite funtion of add(self, other) and neg(other)

In [17]:
def __sub__(self, other):
    return add(self, -other) # Does the add() but with the negative value of other

Node.__sub__ = __sub__

##### Example

In [18]:
x = Node(1)
y = Node(22)
z = x - y
print(z.value) # Composite of ops (First runs the __neg__(self) then does the __sub__(self,other) function)
backward(z)
print(z.grad, x.grad, y.grad)

-21
1 1 -1


#### pow(a, n) — (scalar exponent)

Forward:
- value = x.value ** n (scalar value)
- parents = (x)

Backward:
- d(out)/d(a) = na^(n-1)

In [19]:
# Function to find the power with scalar exponent
def pow(a, n):
    out = Node(
        value= a.value ** n,
        parents= (a,)
    )
    # Gradient Accumulation
    def backward(): # Backward: dy/dx = nx^(n-1)
        a.grad += out.grad * (n * (a.value**(n-1)))
    out.backward_fn= backward
    return out

In [20]:
def __pow__(self, other):
    return pow(self, other)

Node.__pow__ = __pow__

##### Example

In [21]:
x = Node(5)
z = x ** 2
print(z.value)

25


#### exp(a)

Forward:
- value = exp(a.value)
- parents = (a)

Backward:
- d(out)/d(a) = out.value

In [22]:
# Function to find the exponential
def exp(a):
    import math
    out = Node(
        value= math.exp(a.value),
        parents= (a,)
    )
    # Gradient accumulation
    def backward(): # Backward: dy/dx = exp(x) = out.value
        a.grad += out.grad * out.value
    out.backward_fn = backward
    return out

##### Example

In [23]:
x = Node(1)
z = exp(x)
backward(z)
print(z.value, x.grad)  # ≈ 2.71828, 2.71828

2.718281828459045 2.718281828459045


#### log(a)

Forward:
- value = log(a.value)
- parents = (a)

Backward:
- d(out)/d(a) = 1/(a.value)

In [24]:
# Function to find the logarithmic value
def log(a):
    import math
    out = Node(
        value= math.log(a.value),
        parents= (a,)
    )
    # Gradient accumulation
    def backward(): # backward: dy/dx = 1/x
        a.grad += out.grad * (1/a.value)
    out.backward_fn= backward
    return out

##### Example

In [25]:
x = Node(10)
z = log(x)
backward(z)
print(z.value, x.grad)

2.302585092994046 0.1


#### sin(a)

Forward:
- value = sin(a.value)
- parents = (a)

Backward:
- d(out)/d(a) = cos(a.value)

In [26]:
# Function to find sin
def sin(a):
    import math
    out = Node(
        value= math.sin(a.value),
        parents= (a,)
    )
    # Gradient accumulation
    def backward(): # backward: dy/dx = cos(x)
        a.grad += out.grad * math.cos(a.value)
    out.backward_fn = backward
    return out

##### Example

In [27]:
x = Node(5)
z = sin(x)
backward(z)
print(z.value, x.grad)

-0.9589242746631385 0.28366218546322625


### Scalar lifting

Raw scalars are automatically converted into Node's at the operator boundary.

This ensures:
- Engine ops only ever receive Node objects
- No raw scalars leak into the graph

In [28]:
# Function to change the scalar value to a Node object
def lifter(value):
    if isinstance(value, Node): # checks if the value is a node object or not.
        return value
    else:
        return Node(value)

#### Updated ops with lifter function

In [29]:
def __add__(self, other):
    other = lifter(other)
    return add(self, other)

Node.__add__ = __add__

In [30]:
def __mul__(self, other):
    other = lifter(other)
    return mul(self, other)

Node.__mul__ = __mul__

In [31]:
def __sub__(self,other):
    other = lifter(other)
    return add(self, -other)

Node.__sub__ = __sub__

#### Right hand versions (add, mul, sub)

In [32]:
def __radd__(self, other):
    other = lifter(other)
    return add(other, self)

Node.__radd__ = __radd__

In [33]:
def __rmul__(self, other):
    other = lifter(other)
    return mul(other,self)

Node.__rmul__ = __rmul__

In [34]:
def __rsub__(self,other):
    other = lifter(other)
    return add(other, -self)

Node.__rsub__ = __rsub__

## Gradient Checking

To prove correctness, analytical gradients are compared against numerical gradients using finite differences.

- d(f)/d(x) ​≈ (f(x+ϵ)−f(x−ϵ)​)/2ϵ

### Defining f(x)

In [35]:
def f(x):
    return (x + 1) * (x + 1)

### AutoGrad Gradient

In [36]:
def ag_g(f, x0):
    x = Node(x0)
    z = f(x)
    zero_grad(z)
    backward(z)
    return x.grad

### Numerical Gradient

In [37]:
def num_g(f, x0, eps = 1e-4):
    x_pos = Node(x0 + eps)
    x_neg = Node(x0 - eps)

    z_pos = f(x_pos)
    z_neg = f(x_neg)

    return (z_pos.value - z_neg.value) / (2 * eps)

### Gradient Check

- f = Function of the equation
- x0 = Absolute value of x
- eps = epsilon
- tol = tolerance

If the difference of (g_auto - g_num) is less than tolerance then there's no bugs, if not then bugs are present.

In [38]:
def gradcheck(f, x0, eps=1e-4, tol=1e-4):
    g_auto = ag_g(f, x0)
    g_num = num_g(f, x0, eps)

    diff = abs(g_auto - g_num)
    print(f"x = {x0}")
    print(f"autograd grad = {g_auto}")
    print(f"numerical grad = {g_num}")
    print(f"diff = {diff}")

    if diff < tol:
        print("grad check pass")
    else:
        print("grad check fail")

#### Examples

In [39]:
equations = [
    # addition
    ("x + 3",        lambda x: x + 3,        2.0),
    ("3 + x",        lambda x: 3 + x,        2.0),

    # subtraction
    ("x - 3",        lambda x: x - 3,        5.0),
    ("3 - x",        lambda x: 3 - x,        5.0),

    # multiplication
    ("x * 4",        lambda x: x * 4,        3.0),
    ("4 * x",        lambda x: 4 * x,        3.0),

    # power
    ("x ** 2",       lambda x: x ** 2,       2.0),

    # unary
    ("-x",           lambda x: -x,           3.0),

    # nonlinear
    ("exp(x)",       lambda x: exp(x),       1.0),
    ("log(x)",       lambda x: log(x),       2.0),
    ("sin(x)",       lambda x: sin(x),       1.5),
]

In [40]:
composites = [
    ("(x + 1)(x + 1)",   lambda x: (x + 1) * (x + 1),   2.0),
    ("(x - 2)(x + 2)",   lambda x: (x - 2) * (x + 2),   3.0),
    ("(x + 1)(1 + x)",   lambda x: (x + 1) * (1 + x),   2.0),

    ("exp(x) * x",       lambda x: exp(x) * x,          1.0),
    ("log(x) * x",       lambda x: log(x) * x,          2.0),

    ("-(x + 3)",         lambda x: -(x + 3),            4.0),
]

In [42]:
print("\n===== SINGLE OP TESTS =====\n")
for name, fn, x0 in equations:
    print(f"Testing: {name}")
    gradcheck(fn, x0)
    print("-" * 40)

print("\n===== COMPOSITE TESTS =====\n")
for name, fn, x0 in composites:
    print(f"Testing: {name}")
    gradcheck(fn, x0)
    print("-" * 40)


===== SINGLE OP TESTS =====

Testing: x + 3
x = 2.0
autograd grad = 1
numerical grad = 0.9999999999976694
diff = 2.3305801732931286e-12
grad check pass
----------------------------------------
Testing: 3 + x
x = 2.0
autograd grad = 1
numerical grad = 0.9999999999976694
diff = 2.3305801732931286e-12
grad check pass
----------------------------------------
Testing: x - 3
x = 5.0
autograd grad = 1
numerical grad = 0.9999999999976694
diff = 2.3305801732931286e-12
grad check pass
----------------------------------------
Testing: 3 - x
x = 5.0
autograd grad = -1
numerical grad = -0.9999999999976694
diff = 2.3305801732931286e-12
grad check pass
----------------------------------------
Testing: x * 4
x = 3.0
autograd grad = 4
numerical grad = 4.000000000008441
diff = 8.44124770082999e-12
grad check pass
----------------------------------------
Testing: 4 * x
x = 3.0
autograd grad = 4
numerical grad = 4.000000000008441
diff = 8.44124770082999e-12
grad check pass
-------------------------------

## Summary

This project demonstrates the core mechanics of reverse-mode automatic differentiation:
- Dynamic graph construction
- Implicit DAG representation
- Reverse topological traversal
- Gradient accumulation
- Explicit gradient reset
- Numerical verification

This is the foundational mechanism behind modern deep learning frameworks such as PyTorch, JAX, and TensorFlow.